## 🧩 Part 1: Merging Math and Portuguese Datasets

The original data is provided in two separate files — one for **Mathematics (student-math)** and one for **Portuguese (student-por)**.  
There is **no unique student ID**, so we merge the datasets using all **non-grade, non-absences** columns that identify a student (such as school, age, sex, address, family background, etc.).

We excluded subject-specific fields like:  
- `G1_math`, `G2_math`, `G3_math`, `absences_math`  
- `G1_por`, `G2_por`, `G3_por`, `absences_por`

This ensures that only students with exactly matching personal and demographic data are considered the *same student* in both subjects.

If a student exists in only one dataset, they are still kept (via outer join), and missing grades for the other subject are filled as NaN.

The final output is saved as: **`student_combined_data_final.csv`**


In [ ]:
import pandas as pd

# --- Load data ---
math = pd.read_csv("student-mat.csv")
por  = pd.read_csv("student-por.csv")

# --- Grade & subject-specific columns ---
math_subject_cols = ['G1_math', 'G2_math', 'G3_math', 'absences_math']
por_subject_cols  = ['G1_por', 'G2_por', 'G3_por', 'absences_por']

# --- Build strict match key: all shared non-subject, non-grade columns ---
common_cols = sorted(list(set(math.columns).intersection(set(por.columns))))
exclude_cols = set(math_subject_cols + por_subject_cols)
key_cols = [c for c in common_cols if c not in exclude_cols]

# --- Keep only key columns + Portuguese subject columns ---
por_keep = por[key_cols + por_subject_cols].drop_duplicates(subset=key_cols, keep='first')

# --- Merge: all columns from Math + Portuguese subject columns ---
merged = pd.merge(
    math, por_keep,
    on=key_cols,
    how='outer'
)

# --- Diagnostics ---
has_math = merged[['G1_math', 'G2_math', 'G3_math']].notna().any(axis=1)
has_por  = merged[['G1_por', 'G2_por', 'G3_por']].notna().any(axis=1)
matched  = (has_math & has_por).sum()
only_math = (has_math & ~has_por).sum()
only_por  = (~has_math & has_por).sum()

print(f"Matched students (identical on all non-subject columns): {matched}")
print(f"Math-only rows: {only_math}")
print(f"Portuguese-only rows: {only_por}")
print(f"Total rows in merged: {len(merged)}")

# --- Save final combined dataset ---
merged.to_csv("student_combined_data_final.csv", index=False)
print("✅ Merged dataset saved as 'student_combined_data_final.csv'")


Matched students (identical on all non-subject columns): 162
Math-only rows: 233
Portuguese-only rows: 485
Total rows in merged: 880
✅ Merged dataset saved as 'student_combined_data_final.csv'


## 📊 Part 2: Linear Regression Model to Predict Final Math Grade (G3_math)

We now build a **linear regression model** to predict the final Math grade (`G3_math`) using only meaningful non-grade predictors.

### ✅ Why Linear Regression?
- The assignment specifically requires using **linear models**.
- Linear models are **interpretable** — coefficient values show how each feature affects the grade.
- They work well with numeric + one-hot encoded categorical data.

### ✅ Important Rules Applied:
- We **do not use G1 or G2** (previous grades), because the dataset description warns they are highly correlated with G3.
- We only include behavioral, demographic, educational support, and family features.

### ✅ Preprocessing Steps:
1. Identify numerical and categorical features.
2. Use `StandardScaler` to normalize numeric features.
3. Use `OneHotEncoder` for categorical features.
4. Combine both using a `ColumnTransformer`.
5. Fit a `LinearRegression()` model using a pipeline.
6. Check model performance using **Train R²**, **Test R²**, and **5-Fold Cross-Validation R²**.


In [ ]:
# ======================================
# CLEAN LINEAR MODEL FOR G3_math (with only relevant predictors)
# ======================================

import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# --- Load dataset ---
data = pd.read_csv("student_combined_data_final.csv")

# --- Define target ---
target = 'G3_math'

# --- Columns we’ll use (academically meaningful features only) ---
keep_cols = [
    'address','Medu','Fedu','Mjob','Fjob','studytime','failures',
    'schoolsup','famsup','paid','activities','nursery','higher',
    'internet','romantic','famrel','freetime','goout','Dalc','Walc',
    'health','absences_math'
]

# --- Drop rows with missing data in selected columns or target ---
subset = data.dropna(subset=[target] + keep_cols).copy()

# --- Define X and y ---
X = subset[keep_cols]
y = subset[target]

# --- Identify numeric and categorical columns ---
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

# --- Preprocessing: standardize numeric, one-hot encode categorical ---
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols)
])

# --- Linear regression pipeline ---
math_model = Pipeline([
    ('preprocess', preprocessor),
    ('regressor', LinearRegression())
])

# --- Train-test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Fit model ---
math_model.fit(X_train, y_train)

# --- Evaluate ---
train_r2 = math_model.score(X_train, y_train)
test_r2  = math_model.score(X_test, y_test)
cv_r2    = cross_val_score(math_model, X, y, cv=5, scoring='r2').mean()

print("\n📘 Clean Linear Model for G3_math")
print(f"Training R²: {train_r2:.3f}")
print(f"Testing  R²: {test_r2:.3f}")
print(f"5-Fold CV R²: {cv_r2:.3f}")



📘 Clean Linear Model for G3_math
Training R²: 0.283
Testing  R²: 0.044
5-Fold CV R²: 0.088


## 📌 Part 3: Interpreting Feature Importance for Math Performance

After training the model, we extract the **regression coefficients** to see which features have the strongest effect on `G3_math`.

### ✅ How to Interpret Coefficients:
- **Positive coefficient** → Feature increases final grade.
- **Negative coefficient** → Feature decreases final grade.
- Since numeric features are standardized, coefficient magnitudes are comparable.

We sort features by absolute coefficient values to find the **top 15 most influential predictors** of G3 in Math.


In [ ]:
import pandas as pd
import numpy as np

# --- Extract coefficients after preprocessing ---
pre = math_model.named_steps['preprocess']
reg = math_model.named_steps['regressor']

# Numeric + categorical names
num_cols = pre.transformers_[0][2]
cat_encoder = pre.transformers_[1][1]
cat_cols = pre.transformers_[1][2]
cat_names = cat_encoder.get_feature_names_out(cat_cols)

# Combine names and coefficients
feature_names = np.concatenate([num_cols, cat_names])
coefs = reg.coef_

coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefs,
    'AbsCoef': np.abs(coefs)
}).sort_values(by='AbsCoef', ascending=False)

print("\n📊 Top 15 Features Influencing G3_math:")
print(coef_df.head(15).to_string(index=False))



📊 Top 15 Features Influencing G3_math:
      Feature  Coefficient  AbsCoef
   higher_yes     1.864197 1.864197
 Fjob_teacher     1.778002 1.778002
  Mjob_health     1.617405 1.617405
     failures    -1.483045 1.483045
Mjob_services     1.264003 1.264003
schoolsup_yes    -1.176369 1.176369
   famsup_yes    -1.174677 1.174677
        goout    -0.914959 0.914959
 romantic_yes    -0.851519 0.851519
   Fjob_other    -0.790806 0.790806
    address_U     0.649292 0.649292
         Walc     0.549227 0.549227
    studytime     0.540147 0.540147
  Fjob_health     0.539806 0.539806
 Mjob_teacher    -0.521427 0.521427


## 📚 Part 4: Linear Regression Model for Portuguese Grades (G3_por)

We repeat exactly the same process as for Math, but now predicting **`G3_por`**.

### ✅ Why Repeat the Same Model?
- The assignment encourages comparing performance **across both subjects**.
- This helps us answer: *Do the same factors influence Math and Portuguese equally?*
- Evaluation uses the same metrics: Train R², Test R², and 5-Fold CV R².

The only difference is:
- We replace `absences_math` with `absences_por`
- Target variable becomes `G3_por`


In [ ]:
# ======================================
# LINEAR MODEL FOR G3_por (same logic as math)
# ======================================

# --- Define target ---
target = 'G3_por'

# --- Columns we’ll use ---
keep_cols = [
    'address','Medu','Fedu','Mjob','Fjob','studytime','failures',
    'schoolsup','famsup','paid','activities','nursery','higher',
    'internet','romantic','famrel','freetime','goout','Dalc','Walc',
    'health','absences_por'
]

# --- Drop rows with missing data in selected columns or target ---
subset = data.dropna(subset=[target] + keep_cols).copy()

# --- Define X and y ---
X = subset[keep_cols]
y = subset[target]

# --- Identify numeric and categorical columns ---
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

# --- Preprocessing: standardize numeric, one-hot encode categorical ---
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols)
])

# --- Linear regression pipeline ---
por_model = Pipeline([
    ('preprocess', preprocessor),
    ('regressor', LinearRegression())
])

# --- Train-test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Fit model ---
por_model.fit(X_train, y_train)

# --- Evaluate ---
train_r2 = por_model.score(X_train, y_train)
test_r2  = por_model.score(X_test, y_test)
cv_r2    = cross_val_score(por_model, X, y, cv=5, scoring='r2').mean()

print("\n📘 Clean Linear Model for G3_por")
print(f"Training R²: {train_r2:.3f}")
print(f"Testing  R²: {test_r2:.3f}")
print(f"5-Fold CV R²: {cv_r2:.3f}")



📘 Clean Linear Model for G3_por
Training R²: 0.329
Testing  R²: 0.230
5-Fold CV R²: 0.176


## 📌 Part 5: Feature Importance for G3_por

Just like in Math, we extract all feature coefficients from the Portuguese regression model.

### ✅ What We Look For:
- Which features have the strongest positive or negative impact on Portuguese scores?
- Do the same variables (like study time, failures, parental education) affect both subjects?
- Are there differences in how social behavior or family structure influences performance in language vs math?

The output shows the **top 15 most influential features** for `G3_por`.


In [ ]:
import pandas as pd
import numpy as np

# --- Extract coefficients for Portuguese model ---
pre = por_model.named_steps['preprocess']
reg = por_model.named_steps['regressor']

# Numeric + categorical names
num_cols = pre.transformers_[0][2]
cat_encoder = pre.transformers_[1][1]
cat_cols = pre.transformers_[1][2]
cat_names = cat_encoder.get_feature_names_out(cat_cols)

# Combine names and coefficients
feature_names = np.concatenate([num_cols, cat_names])
coefs = reg.coef_

coef_por = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefs,
    'AbsCoef': np.abs(coefs)
}).sort_values(by='AbsCoef', ascending=False)

print("\n📊 Top 15 Features Influencing G3_por:")
print(coef_por.head(15).to_string(index=False))



📊 Top 15 Features Influencing G3_por:
      Feature  Coefficient  AbsCoef
   higher_yes     1.710989 1.710989
schoolsup_yes    -1.093523 1.093523
     failures    -0.899637 0.899637
  Mjob_health     0.691130 0.691130
    address_U     0.666919 0.666919
 internet_yes     0.618528 0.618528
     paid_yes    -0.592236 0.592236
  Fjob_health     0.589793 0.589793
 Fjob_teacher     0.553384 0.553384
    studytime     0.485899 0.485899
Fjob_services    -0.451688 0.451688
 romantic_yes    -0.408490 0.408490
Mjob_services     0.383790 0.383790
       health    -0.305687 0.305687
  nursery_yes    -0.295233 0.295233
